Imports:


In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin # Para definição de transformadores personalizados
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 1. Introdução

Carregando dados:

In [36]:
df_train = pd.read_csv('./data/train.csv', parse_dates=['activation_date'])

In [37]:
df_train.head(10)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,Private,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,796.0,0.80323
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823.0,0.00000
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,Private,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,567.0,0.80323
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85,2017-03-17,Private,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,415.0,0.00000
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,Company,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,46.0,0.00000


In [44]:
df_train.columns

Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'activation_date',
       'user_type', 'image', 'image_top_1', 'deal_probability'],
      dtype='object')

**TODO**: Visualizações de dados cairiam bem aqui. Consultar livro do Geron

# 2. Processando dados

In [17]:
print('Nº. de amostras:', len(df_train.index))

Nº. de amostras: 1503424


Justificar, com base nessa primeira olhada nos dados, as trasformações de preprocessamento a serem realizadas: imputação de dados faltantes, vetorização de dados textuais, redução de dimensionalidade com PCA (ou eliminação direta, perguntar ao Fabrício), normalização (apenas para os algoritmos que se beneficiam)

A partir dessa primeira exploração dos dados, podemos ver que o dataset é bastante heterogêneo. Nele, podemos observar features categóricas, numéricas e textuais, o que significa que teremos que executar algumas etapas de preprocessamento antes de treinar algum modelo.

## 2.1 Codificando features categóricas

Contagem de classes nas features categóricas:

In [18]:
print('Regions:', len(np.unique(df_train['region'])))
print('cities:', len(np.unique(df_train['city'])))
print('parent_category_names:', len(np.unique(df_train['parent_category_name'])))
print('category_names:', len(np.unique(df_train['category_name'])))
# print('param_1:', len(np.unique(df_train['param_1'])))
# print('param_2:', len(np.unique(df_train['param_2'])))
# print('param_3:', len(np.unique(df_train['param_3'])))
print('activation_dates:', len(np.unique(df_train['activation_date'])))
print('user_types:', len(np.unique(df_train['user_type'])))
print('image_top_1:', len(np.unique(df_train['image_top_1'])))

Regions: 28
cities: 1733
parent_category_names: 9
category_names: 47
activation_dates: 21
user_types: 3
image_top_1: 115650


As colunas *region*, *city*, *parent_category_name*, *category_name* e *user_type* são features categóricas nominais. Isso significa que, para cada amostra, essas entradas assumem um dentre um número finito de classes possíveis. Além disso, não existe uma ordem entre as possíveis classes de uma dessas features. Uma forma óbvia para codificar numericamente uma feature categórica é substituir cada classe dessa feature por um número inteiro. Entretanto, se fizésssemos isso, estaríamos inserindo uma falsa informação de ordem entre as classes. Uma técnica muito comum para lidar com essa situação é o One-Hot Encoding, onde uma coluna é criada para cada classe de uma feature categórica: 

In [24]:
df_train = pd.get_dummies(
    df_train,
    columns=['region', 'city', 'parent_category_name', 'category_name', 'user_type'],
    drop_first=True
)

In [28]:
print(df_train.shape)

(1503424, 1828)


O aumento colossal no número de features é problemático

~~Essa operação exige bastante do computador, então vou gerar um CSV nesse ponto para não ter que repetí-la~~ O arquivo final seria gigantesco. Inviável

In [27]:
# df_train.to_csv('./data/train-one_hot.csv')

KeyboardInterrupt: 

In [30]:
df_train.head(10)

,item_id,user_id,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,...,category_name_Собаки,category_name_Спорт и отдых,category_name_Телефоны,category_name_Товары для детей и игрушки,category_name_Товары для животных,category_name_Товары для компьютера,category_name_Фототехника,category_name_Часы и украшения,user_type_Private,user_type_Shop
0,b912c3c6a6ad,e00f8ff2eaf9,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,...,0,0,0,1,0,0,0,0,1,0
1,2dac0150717d,39aeb48f0017,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,...,0,0,0,0,0,0,0,0,1,0
2,ba83aefab5dc,91e2f88dd6e3,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,...,0,0,0,0,0,0,0,0,1,0
3,02996f1dd2ea,bf5cccea572d,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,...,0,0,0,1,0,0,0,0,0,0
4,7c90be56d2ab,ef50846afc0b,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,...,0,0,0,0,0,0,0,0,1,0
5,51e0962387f7,bbfad0b1ad0a,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,...,0,0,0,1,0,0,0,0,1,0
6,c4f260a2b48a,08f469d2e6f7,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,...,0,0,0,0,0,0,0,0,1,0
7,6b71309d6a8a,fef86baa002c,Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,...,0,0,0,0,0,0,0,0,1,0
8,c5b969cb63a2,055825270190,Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85,2017-03-17,...,0,0,0,0,0,0,0,0,1,0
9,b1570962e68c,f9e8f831d94c,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,...,0,0,0,0,0,0,0,0,0,0


**Atenção**: Aparentemente, fazer o One Hot Encoding é bem estúpido, pois não faz proveito das otimizações de matrizes esparsas do SciPy. Acho que deveria usar dataframes apenas para carregamento dos dados e após, usar apenas classes do numpy e scipy

Vamos criar um transformador personalizado para codificar essas categorias:

In [66]:
# Atenção! Se você aplicar um transformador de remoção de features antes desse, esses índices terão de mudar!
cat_indexes = [2, 3, 4, 5, 14]

class CategoricalFeaturesEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, use_one_hot=False):
        self.les = {i: LabelEncoder() for i in cat_indexes}
        self.ohe = None
        if use_one_hot == True:
            self.ohe = OneHotEncoder(categorical_features=cat_indexes) 
        
    def fit(self, X, y=None):
        for cat_idx in cat_indexes:
            self.les[cat_idx].fit(X[:, cat_idx])
        return self
    
    def transform(self, X, y=None):
        X_transf = X.copy()
        for cat_idx in cat_indexes:
            X_transf[:, cat_idx] = self.les[cat_idx].transform(X_transf[:, cat_idx])
            
        if self.ohe != None:
            print('One Hot')
            X_transf = self.ohe.fit_transform(X_transf)
            
        return X_transf

Testando transformador:

In [49]:
transf = CategoricalFeaturesEncoder()
X_transf = transf.fit_transform(df_train.values)

In [50]:
X_transf[:10]

array([['b912c3c6a6ad', 'e00f8ff2eaf9', 19, 460, 4, 42,
        'Постельные принадлежности', nan, nan, 'Кокоби(кокон для сна)',
        'Кокон для сна малыша,пользовались меньше месяца.цвет серый',
        400.0, 2, Timestamp('2017-03-28 00:00:00'), 1,
        'd10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c7679f17c333c959b19',
        1008.0, 0.12789],
       ['2dac0150717d', '39aeb48f0017', 17, 1300, 2, 22, 'Другое', nan,
        nan, 'Стойка для Одежды',
        'Стойка для одежды, под вешалки. С бутика.', 3000.0, 19,
        Timestamp('2017-03-26 00:00:00'), 1,
        '79c9392cc51a9c81c6eb91eceb8e552171db39d71427001e1e0859c0fb843a31',
        692.0, 0.0],
       ['ba83aefab5dc', '91e2f88dd6e3', 16, 1276, 0, 2,
        'Видео, DVD и Blu-ray плееры', nan, nan, 'Philips bluray',
        'В хорошем состоянии, домашний кинотеатр с blu ray, USB. Если настроить, то работает смарт тв /\nТорг',
        4000.0, 9, Timestamp('2017-03-20 00:00:00'), 1,
        'b7f250ee3f39e1fedd77c141f273703f

In [51]:
df_transf = pd.DataFrame(X_transf, columns=df_train.columns)

In [52]:
df_transf.head(10)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,19,460,4,42,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400,2,2017-03-28 00:00:00,1,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008,0.12789
1,2dac0150717d,39aeb48f0017,17,1300,2,22,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000,19,2017-03-26 00:00:00,1,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692,0
2,ba83aefab5dc,91e2f88dd6e3,16,1276,0,2,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000,9,2017-03-20 00:00:00,1,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032,0.43177
3,02996f1dd2ea,bf5cccea572d,21,940,4,42,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200,286,2017-03-25 00:00:00,0,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796,0.80323
4,7c90be56d2ab,ef50846afc0b,4,317,6,0,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000,3,2017-03-16 00:00:00,1,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264,0.20797
5,51e0962387f7,bbfad0b1ad0a,21,1657,4,42,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300,9,2017-03-28 00:00:00,1,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,796,0.80323
6,c4f260a2b48a,08f469d2e6f7,11,975,2,38,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000,125,2017-03-23 00:00:00,1,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823,0
7,6b71309d6a8a,fef86baa002c,15,1153,4,29,Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500,61,2017-03-25 00:00:00,1,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,567,0.80323
8,c5b969cb63a2,055825270190,14,1107,4,29,Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500,85,2017-03-17 00:00:00,1,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,415,0
9,b1570962e68c,f9e8f831d94c,11,975,4,10,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400,136,2017-03-22 00:00:00,0,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,46,0


Testando, agora, com One Hot Encoding:

In [67]:
transf = CategoricalFeaturesEncoder(use_one_hot=True)
X_transf = transf.fit_transform(df_train.values)

One Hot


ValueError: could not convert string to float: '87cca5889b96b0368bffbe7cdb3f0cc5fa660f33be9fd027d0fc95dc15cdcb3e'

In [61]:
X_transf.shape

(1503424, 18)

In [55]:
X_transf[:10]

array([['b912c3c6a6ad', 'e00f8ff2eaf9', 19, 460, 4, 42,
        'Постельные принадлежности', nan, nan, 'Кокоби(кокон для сна)',
        'Кокон для сна малыша,пользовались меньше месяца.цвет серый',
        400.0, 2, Timestamp('2017-03-28 00:00:00'), 1,
        'd10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c7679f17c333c959b19',
        1008.0, 0.12789],
       ['2dac0150717d', '39aeb48f0017', 17, 1300, 2, 22, 'Другое', nan,
        nan, 'Стойка для Одежды',
        'Стойка для одежды, под вешалки. С бутика.', 3000.0, 19,
        Timestamp('2017-03-26 00:00:00'), 1,
        '79c9392cc51a9c81c6eb91eceb8e552171db39d71427001e1e0859c0fb843a31',
        692.0, 0.0],
       ['ba83aefab5dc', '91e2f88dd6e3', 16, 1276, 0, 2,
        'Видео, DVD и Blu-ray плееры', nan, nan, 'Philips bluray',
        'В хорошем состоянии, домашний кинотеатр с blu ray, USB. Если настроить, то работает смарт тв /\nТорг',
        4000.0, 9, Timestamp('2017-03-20 00:00:00'), 1,
        'b7f250ee3f39e1fedd77c141f273703f

In [57]:
df_transf = pd.DataFrame(X_transf, columns=df_train.columns)

In [58]:
df_transf.head(10)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,19,460,4,42,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400,2,2017-03-28 00:00:00,1,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008,0.12789
1,2dac0150717d,39aeb48f0017,17,1300,2,22,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000,19,2017-03-26 00:00:00,1,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692,0
2,ba83aefab5dc,91e2f88dd6e3,16,1276,0,2,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000,9,2017-03-20 00:00:00,1,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032,0.43177
3,02996f1dd2ea,bf5cccea572d,21,940,4,42,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200,286,2017-03-25 00:00:00,0,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796,0.80323
4,7c90be56d2ab,ef50846afc0b,4,317,6,0,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000,3,2017-03-16 00:00:00,1,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264,0.20797
5,51e0962387f7,bbfad0b1ad0a,21,1657,4,42,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300,9,2017-03-28 00:00:00,1,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,796,0.80323
6,c4f260a2b48a,08f469d2e6f7,11,975,2,38,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000,125,2017-03-23 00:00:00,1,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823,0
7,6b71309d6a8a,fef86baa002c,15,1153,4,29,Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500,61,2017-03-25 00:00:00,1,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,567,0.80323
8,c5b969cb63a2,055825270190,14,1107,4,29,Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500,85,2017-03-17 00:00:00,1,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,415,0
9,b1570962e68c,f9e8f831d94c,11,975,4,10,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400,136,2017-03-22 00:00:00,0,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,46,0


In [4]:
df_train.isnull().sum()

item_id                      0
user_id                      0
region                       0
city                         0
parent_category_name         0
category_name                0
param_1                  61576
param_2                 654542
param_3                 862565
title                        0
description             116276
price                    85362
item_seq_number              0
activation_date              0
user_type                    0
image                   112588
image_top_1             112588
deal_probability             0
dtype: int64

param_1, param_2 e param_3 são parâmetros adicionais de classificação do modelo de anúncios da plataforma. Podem não ser muito essenciais. image é o identificador do jpeg associado ao anúncio. Como não analizaremos imagens, parece seguro ignorar essa feature. image_top_1 é uma incógnita. Não sei seu significado. Como é relacionado com imagens, irei ignorar. Talvez *item_seq_number* também seja ignorável.

In [31]:
df_train.drop(columns=['param_1', 'param_2', 'param_3' , 'image_top_1'], inplace=True)

In [33]:
df_train.drop(columns=['item_id', 'user_id', 'title', 'description', 'item_seq_number', 'activation_date', 'image'], inplace=True)

In [34]:
df_train.head(10)

,price,deal_probability,region_Башкортостан,region_Белгородская область,region_Владимирская область,region_Волгоградская область,region_Воронежская область,region_Иркутская область,region_Калининградская область,region_Кемеровская область,...,category_name_Собаки,category_name_Спорт и отдых,category_name_Телефоны,category_name_Товары для детей и игрушки,category_name_Товары для животных,category_name_Товары для компьютера,category_name_Фототехника,category_name_Часы и украшения,user_type_Private,user_type_Shop
0,400.0,0.12789,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,3000.0,0.00000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4000.0,0.43177,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2200.0,0.80323,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,40000.0,0.20797,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,1300.0,0.80323,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
6,11000.0,0.00000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,500.0,0.80323,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,500.0,0.00000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,400.0,0.00000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


O que fazer quanto as datas?

In [6]:
df_train.activation_date.values[:10]

array(['2017-03-28', '2017-03-26', '2017-03-20', '2017-03-25',
       '2017-03-16', '2017-03-28', '2017-03-23', '2017-03-25',
       '2017-03-17', '2017-03-22'], dtype=object)

In [9]:
dates = pd.DatetimeIndex(df_train.activation_date.values)

In [10]:
dates

DatetimeIndex(['2017-03-28', '2017-03-26', '2017-03-20', '2017-03-25',
               '2017-03-16', '2017-03-28', '2017-03-23', '2017-03-25',
               '2017-03-17', '2017-03-22',
               ...
               '2017-03-24', '2017-03-15', '2017-03-24', '2017-03-27',
               '2017-03-17', '2017-03-20', '2017-03-28', '2017-03-21',
               '2017-03-22', '2017-03-21'],
              dtype='datetime64[ns]', length=1503424, freq=None)

In [8]:
dates.year.max()

2017

In [9]:
dates.year.min()

2017

In [16]:
dates.month.min()

3

In [17]:
dates.month.max()

4

Todas as datas são de 2017. Poderia projetá-las em 12 categorias representando os meses ou talvez uma categorização ainda mais fina

Não sei se existe uma técnica de imputação para lidar com dados textuais faltantes. Talvez tenhamos que eliminar as amostras que não tenham descrição.

Imputações deveriam ser feitas antes ou depois de eliminações de amostras?

Não gostaria de ignorar a feature 'preço', pois me parece extremamente relevante para o sucesso do anúncio. Nesse caso, uma imputação da média pode ser interessante:

In [10]:
df_train[['price']]

,price
0,400.0
1,3000.0
2,4000.0
3,2200.0
4,40000.0
5,1300.0
6,11000.0
7,500.0
8,500.0
9,400.0


In [11]:
from sklearn.preprocessing import Imputer

imr = Imputer(missing_values='NaN', strategy='mean', axis=0)

imr.fit(df_train[['price']].values)
imputed_prices = imr.transform(df_train[['price']].values)

df_train.price = imputed_prices

In [12]:
df_train.head(10)

,item_id,user_id,region,city,parent_category_name,category_name,title,description,price,item_seq_number,activation_date,user_type,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,0.20797
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,Private,0.80323
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0.00000
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,Private,0.80323
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Платье,54 раз мер очень удобное,500.0,85,2017-03-17,Private,0.00000
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,Company,0.00000


In [13]:
df_train.isnull().sum()

item_id                      0
user_id                      0
region                       0
city                         0
parent_category_name         0
category_name                0
title                        0
description             116276
price                        0
item_seq_number              0
activation_date              0
user_type                    0
deal_probability             0
dtype: int64

**Lembrar de aplicar transformações também aos dados de teste, MAS O TRANSFORMADOR DEVE SER TREINADO APENAS COM OS DADOS DE TREINO**

Agora, é preciso codificar as features categóricas nominais: region, city, parent_category_name e user_type. Provavelmente teremos de usar one-hot encoding